## Tensorflow model

In [814]:
# Import packages for tensorflow
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM, LeakyReLU, BatchNormalization, Conv1D, MaxPooling1D, Activation
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint

In [815]:
plt.style.use('fivethirtyeight')
plt.ioff()

In [816]:
var_set = pd.read_csv('var_set.csv', parse_dates=[0],index_col=[0])

In [817]:
var_set

,SalmonPrice,CodPrice,HalibutPrice,CPI,TWI
2013-04-07,39.63,10.754267,47.577612,95.55,92.1500
2013-04-14,41.89,10.800751,47.924958,95.70,92.1920
2013-04-21,43.07,10.774089,47.092639,95.85,92.7980
2013-04-28,42.70,10.569605,47.193296,96.00,93.8080
2013-05-05,41.81,10.605343,48.987917,96.02,93.3125
...,...,...,...,...,...
2022-11-20,82.49,39.417280,76.897925,128.50,122.2180
2022-11-27,79.03,39.071706,73.642666,128.40,122.0320
2022-12-04,78.05,42.701713,73.193629,128.70,120.8780
2022-12-11,75.83,43.652968,74.156841,129.00,122.7060


In [818]:
X = var_set.drop(columns=["SalmonPrice"], axis = 1)
y = var_set["SalmonPrice"]

In [819]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 50, shuffle=False)

In [820]:
x_uni = var_set["SalmonPrice"]

In [821]:
x_uni_train,y_uni_test = train_test_split(x_uni, test_size = 50, shuffle=False)

In [822]:
len(X_train)
y_train.shape

(457,)

In [823]:
# # Make predictions
# test_predictions = []

# first_eval_batch = x_minmax_train[-52:]
# current_batch = first_eval_batch.reshape((1, 52, 1))

# for i in range(len(y_minmax_test)):
#     current_pred = model.predict(current_batch)[0]
#     test_predictions.append(current_pred)
#     current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

# true_predictions = scaler.inverse_transform(test_predictions)

# # Plot the predictions
# test_predictions = pd.Series(true_predictions.reshape(50,))
# y_uni_test = pd.Series(y_uni_test.reshape(50,))
# plt.plot(y_uni_test, label='Test Data')
# plt.plot(test_predictions, label='Predictions')
# plt.legend()

# # Evaluate the model
# from sklearn.metrics import mean_squared_error
# from math import sqrt

# rmse = sqrt(mean_squared_error(y_uni_test, test_predictions))
# print('Test RMSE: %.3f' % rmse)

In [824]:
# Scale x and y multivariate
scaler_x = MinMaxScaler()
X_train = scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)

# Scale y
scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.values.reshape(-1,1))
y_test = scaler_y.transform(y_test.values.reshape(-1,1))

In [825]:
X_train.shape

(457, 4)

In [826]:
y_train.shape

(457, 1)

In [827]:
#from tensorflow.keras.optimizers.legacy import Adam
batch_size = [1, 2, 13, 26, 52, 104] #i

epochs = [10, 40, 100]
#epochs = [1, 2, 3]

rmse_list = []

n_input = [1, 4, 52, 104] #n
#n_input = [24, 52] #n

run = 0
b = 0
e = 0

m1 = 0
m2 = 0

uni_multi = [X_train, y_train]

optimuzer = ['adam', 'nadam']

rmse_df = pd.DataFrame(columns=['Run', 'Batch Size', 'Epochs'])
for i in batch_size:
    for opt in optimuzer:
        for n in n_input:
            b = b + 1
            for j in epochs:
                for m1 in range(1):
                    for vari in uni_multi:
                        e = e + 1
                        print('Batch size: '+str(i))
                        print('Lookback -- input: '+str(n))
                        print('Optimizer: '+str(opt))
                        print('Epochs: '+str(j))
                        
                        if run % 2 == 0:
                            uni_name = 'Univariate'
                        else:
                            uni_name = 'Multivariate'
                        print(uni_name)
                        run = run + 1
                        print('Run: '+str(run))
                        generated_batches = TimeseriesGenerator(vari, y_train, length=n, batch_size=1)
                        model_name_string = 'Model'+ ' ' + str(run)
                        model_name = 'model_name' + str(run)
                        model_name_test = 'model_name' + str(run)
                        print(model_name)
                        model_name = Sequential()
                        model_name.add(Flatten(input_shape=(n,1)))
                        #model_name.add(Dense(1024, activation='relu', name="hidden_layer_1"))
                        if m1 == 1:
                            model_name.add(Dense(128, activation='relu'))
                            model_name.add(Dropout(0.1))
                            model_name.add(Dense(64, activation='relu'))
                            model_name.add(Dropout(0.1))
                        #m1 = m1 + 1
                        model_name.add(Dense(32, activation='relu'))
                        model_name.add(Dense(1))
                        model_name.compile(optimizer=opt, loss='mse')
                        history = 'history'+str(run)
                        history = model_name.fit(generated_batches,epochs=j,verbose=0, batch_size=i)
                        #print(model_name.summary())
                        model_name_test = model_name
                        test_predictions = []
                        first_eval_batch = y_train[-n:]
                        current_batch = first_eval_batch.reshape((1, n, 1))
                        print(current_batch)
                        for k in range(len(y_test)):
                            current_pred = model_name.predict(current_batch)[0]
                            test_predictions.append(current_pred)
                            current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
                        true_predictions = scaler_y.inverse_transform(test_predictions)
                        #print(test_predictions)
                        
                        y_test_inverse = scaler_y.inverse_transform(y_test)
                        rmse = sqrt(mean_squared_error(y_test_inverse, true_predictions))
                        rmse_df = rmse_df.append({'Model Name' : model_name_string, 'Run': run, 'Model alternative':m1+1, 'Batch Size': i, 'Epochs': j, 'Optimizer' : opt, 'Time steps':n, 'Uni/Multi' :uni_name, 'RMSE': rmse}, ignore_index=True)
                        rmse_list.append(rmse)
                        print('Test RMSE: %.3f' % rmse)
                        #mse = mean_squared_error(y_test_inverse, true_predictions)
                        #print('Test MSE: %.3f' % mse)
                        plot_predictions = pd.Series(true_predictions.reshape(50,))
                        y_test_inverse = pd.Series(y_test_inverse.reshape(50,))
                        plt.figure(figsize=(10, 6))
                        plt.plot(history.history['loss'], label='Loss(MSE)')
                        plt.legend()
                        plt.savefig('Figures/Neural networks/ForLoop_Tensor/plotLoss_'+str(run)+'.png')

                        plt.figure(figsize=(18,10))
                        plt.plot(plot_predictions, label='Forecast')
                        plt.plot(y_test_inverse, label='Actual')
                        plt.legend()
                        plt.savefig('Figures/Neural networks/ForLoop_Tensor/plotModel_'+str(run)+'.png')

Batch size: 1
Lookback -- input: 1
Optimizer: adam
Epochs: 10
Univariate
Run: 1
model_name1


2023-04-25 18:19:13.451202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[[[0.82496224]]]
1/1 [==============================] - 0s 11ms/step


2023-04-25 18:19:33.098996: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 10ms/step
Test RMSE: 37.943
Batch size: 1
Lookback -- input: 1
Optimizer: adam
Epochs: 10
Multivariate
Run: 2
model_name2


...

/var/folders/xy/_0zst1c12b9_m_438bw49zw40000gn/T/ipykernel_74781/3651584601.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rmse_df = rmse_df.append({'Model Name' : model_name_string, 'Run': run, 'Model alternative':m1+1, 'Batch Size': i, 'Epochs': j, 'Optimizer' : opt, 'Time steps':n, 'Uni/Multi' :uni_name, 'RMSE': rmse}, ignore_index=True)
2023-04-26 04:09:41.881070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[[[0.96865559]
  [0.94391994]
  [0.80003776]
  [0.70675982]
  [0.65445619]
  [0.75962991]
  [0.88821752]
  [0.7543429 ]
  [0.7586858 ]
  [0.5989426 ]
  [0.53096677]
  [0.48319486]
  [0.49188066]
  [0.47885196]
  [0.47715257]
  [0.42258308]
  [0.40804381]
  [0.5477719 ]
  [0.74905589]
  [0.73017372]
  [0.80947885]
  [0.85536254]
  [0.64558157]
  [0.57024169]
  [0.60064199]
  [0.50660876]
  [0.38651813]
  [0.33836858]
  [0.32024169]
  [0.32061934]
  [0.39010574]
  [0.39935801]
  [0.3489426 ]
  [0.34856495]
  [0.36839124]
  [0.33553625]
  [0.39935801]
  [0.36291541]
  [0.30570242]
  [0.35970544]
  [0.31476586]
  [0.26812689]
  [0.2777568 ]
  [0.27794562]
  [0.33761329]
  [0.28512085]
  [0.25245468]
  [0.2603852 ]
  [0.29361782]
  [0.35630665]
  [0.34762085]
  [0.29361782]
  [0.37084592]
  [0.33572508]
  [0.32533988]
  [0.40861027]
  [0.40766616]
  [0.5173716 ]
  [0.6944864 ]
  [0.66333082]
  [0.62688822]
  [0.69335347]
  [0.79361782]
  [0.74811178]
  [0.61688066]
  [0.61046073]
  [0.71034

2023-04-26 04:14:14.522373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 9ms/step
Test RMSE: 23.942


/var/folders/xy/_0zst1c12b9_m_438bw49zw40000gn/T/ipykernel_74781/3651584601.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rmse_df = rmse_df.append({'Model Name' : model_name_string, 'Run': run, 'Model alternative':m1+1, 'Batch Size': i, 'Epochs': j, 'Optimizer' : opt, 'Time steps':n, 'Uni/Multi' :uni_name, 'RMSE': rmse}, ignore_index=True)


In [828]:
# y_test = scaler_y.inverse_transform(y_test)
# plot_predictions = pd.Series(true_predictions.reshape(50,))
# y_test = pd.Series(y_test.reshape(50,))
# plt.plot(y_test, label='Test Data')
# plt.plot(plot_predictions, label='Predictions')
# plt.legend()

In [829]:
rmse_list

[37.94271142378444,
 31.080594798791296,
 31.504919583473136,
 27.28607984348641,
 33.27083944592211,
 30.46799717695654,
 33.72774206393172,
 27.18405529997446,
 30.977470916565363,
 20.75511073056706,
 32.97677518838975,
 24.530625697483543,
 29.749529969211174,
 26.470382131220482,
 29.500530705318184,
 32.442273481601504,
 30.416164336328457,
 21.991388353563917,
 25.799270008794057,
 35.100582683100356,
 35.27914648582272,
 24.49986705826221,
 34.834384324157696,
 33.45264759525933,
 36.71215473738198,
 28.194334639504753,
 32.553942621308174,
 29.82908259698817,
 32.5333914297123,
 34.019807445671624,
 32.44471596170372,
 36.14313503812986,
 31.525045089046777,
 34.765215901191105,
 31.91751332698815,
 22.71394672178949,
 35.51963336526689,
 31.34325084503235,
 32.97629330290664,
 32.971650655864764,
 30.2245303803617,
 22.176932934612207,
 29.81507987749219,
 38.58210036323327,
 35.22545687548303,
 17.2501465449454,
 35.78049366899354,
 30.609339512603604,
 33.60346307726833,
 3

In [830]:
true_predictions

array([[70.42823687],
       [72.44862996],
       [72.37753246],
       [71.10923303],
       [70.48238625],
       [70.27769881],
       [69.62611643],
       [66.95790685],
       [64.60649355],
       [64.42795275],
       [64.32645659],
       [61.752748  ],
       [57.52731603],
       [61.57475646],
       [67.75137172],
       [68.91451643],
       [71.48053539],
       [75.50793102],
       [75.55901213],
       [73.73206875],
       [74.15415549],
       [75.06388022],
       [72.60305702],
       [68.49861674],
       [66.867358  ],
       [67.08708061],
       [66.38383945],
       [62.92124957],
       [60.27739679],
       [60.40559515],
       [57.84413756],
       [53.0282709 ],
       [52.46299797],
       [51.96273649],
       [49.86554812],
       [49.42619128],
       [51.26882958],
       [52.56529749],
       [54.19106523],
       [57.79585958],
       [57.6543843 ],
       [54.62643205],
       [57.39527286],
       [62.17748003],
       [63.65767672],
       [63

In [831]:
y_test_inverse

0      71.52
1      70.81
2      73.48
3      80.66
4      92.82
5      91.41
6      85.35
7      82.96
8      80.61
9      86.38
10     88.64
11    102.78
12    109.91
13    115.17
14    125.87
15    122.75
16    104.69
17     99.34
18    102.89
19    101.13
20    107.09
21    106.90
22    105.15
23     98.30
24     95.84
25     92.30
26     79.14
27     70.89
28     72.56
29     71.69
30     65.78
31     59.56
32     57.15
33     60.74
34     62.40
35     62.88
36     63.23
37     68.21
38     71.80
39     70.86
40     69.96
41     67.62
42     66.20
43     70.70
44     80.64
45     82.49
46     79.03
47     78.05
48     75.83
49     86.64
dtype: float64

In [832]:
rmse_df['Epochs'] = rmse_df['Epochs'].astype(int)
rmse_df['Batch Size'] = rmse_df['Batch Size'].astype(int)
rmse_df['Run'] = rmse_df['Run'].astype(int)
rmse_df['Model alternative'] = rmse_df['Model alternative'].astype(int)
rmse_df['Time steps'] = rmse_df['Time steps'].astype(int)

In [833]:
rmse_df.insert(8, 'RMSE', rmse_df.pop('RMSE'))

In [834]:
rmse_df

,Run,Batch Size,Epochs,Model Name,Model alternative,Optimizer,Time steps,Uni/Multi,RMSE
0,1,1,10,Model 1,1,adam,1,Univariate,37.942711
1,2,1,10,Model 2,1,adam,1,Multivariate,31.080595
2,3,1,40,Model 3,1,adam,1,Univariate,31.504920
3,4,1,40,Model 4,1,adam,1,Multivariate,27.286080
4,5,1,100,Model 5,1,adam,1,Univariate,33.270839
...,...,...,...,...,...,...,...,...,...
283,284,104,10,Model 284,1,nadam,104,Multivariate,24.205372
284,285,104,40,Model 285,1,nadam,104,Univariate,38.613904
285,286,104,40,Model 286,1,nadam,104,Multivariate,17.215180
286,287,104,100,Model 287,1,nadam,104,Univariate,32.710079


In [835]:
rmse_df.style.to_latex("Figures/Neural networks/RMSE_Tensorflow.tex")

In [836]:
rmse_df.to_csv('Figures/Neural networks/RMSE_Tensorflow.csv')

In [837]:
rmse_df.sort_values(by=['RMSE']).head(20).style.to_latex("Figures/Neural networks/RMSE_Tensorflow_top20.tex")

In [838]:
# pred_list = []

# batch = X_train[-n_input:].reshape((1, n_input, 1))

# for i in range(n_input):   
#     pred_list.append(model_name.predict(batch)[0]) 
#     batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [839]:
# pred_list = scaler_x.inverse_transform(pred_list)

In [840]:
# plt.plot(pred_list, label='Predictions')
# plt.plot(scaler_x.inverse_transform(X_test), label='True Data')

In [841]:
# from pandas.tseries.offsets import DateOffset
# df = pd.DataFrame()
# add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,13) ]
# future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [842]:
# rmse_df.sort_values(by=['RMSE'], ascending=True)

In [843]:
# rmse_df